In [2]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [3]:
# Extract Time
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [4]:
# Find Authors or Contacts
def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

In [5]:
# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [13]:
import os

In [16]:
os.chdir(r"C:\Downloads")

In [24]:
os.listdir()

['books',
 'dataset.xlsx',
 'deep learning',
 'Downloads - Shortcut.lnk',
 'hadoop',
 'html  notes',
 'ml',
 'ml files',
 'new data populayion',
 'New folder',
 'New folder (3)',
 'nlp',
 'projectair',
 'projectair.pkl',
 'R',
 'spark',
 'tablue',
 'Telegram Desktop',
 'time series',
 'WALLPAPER',
 'WhatsApp Chat with The boys.txt']

In [28]:
data = []
with open('WhatsApp Chat with The boys.txt', encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [29]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])

data = df.dropna()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]
print(data.head())

C:\Users\Sonu sharma\AppData\Local\Temp\ipykernel_11988\3707859296.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
C:\Users\Sonu sharma\AppData\Local\Temp\ipykernel_11988\3707859296.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]


        Date     Time      Author          Message  Positive  Negative  \
5 2023-07-02  4:05 pm  Vimal Bhai  <Media omitted>       0.0       0.0   
6 2023-07-02  4:06 pm  Vimal Bhai  <Media omitted>       0.0       0.0   
7 2023-07-02  4:06 pm  Vimal Bhai  <Media omitted>       0.0       0.0   
8 2023-07-02  4:06 pm  Vimal Bhai  <Media omitted>       0.0       0.0   
9 2023-07-02  4:12 pm  Vimal Bhai  <Media omitted>       0.0       0.0   

   Neutral  
5      1.0  
6      1.0  
7      1.0  
8      1.0  
9      1.0  


C:\Users\Sonu sharma\AppData\Local\Temp\ipykernel_11988\3707859296.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]


In [30]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def sentiment_score(a, b, c):
    if (a>b) and (a>c):
        print("Positive 😊 ")
    elif (b>a) and (b>c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")
sentiment_score(x, y, z)

Neutral 🙂 
